SN7

In [11]:
import os
import numpy as np 
import rasterio
import torch
import cv2
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from glob import glob
import geopandas as gpd
from rasterio.features import rasterize
import matplotlib.pyplot as plt


Funzione per calcolare media e deviazione standard

In [5]:
def compute_mean_std(image_paths):
    means = []  
    stds = []  
    
    for img_path in image_paths:
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)  # Carica l'immagine inclusi tutti i canali
        img_rgb = img[:, :, :3]  # Prendo i primi tre canali (RGB)
        #permute mi serve perchè img la leggo con opencv che mi da HxWxC e pytorch vuole CxHxW
        img_tensor = torch.tensor(img_rgb, dtype=torch.float32).permute(2, 0, 1)  # Converti in tensor e permuta le dimensioni a (Canali, Altezza, Larghezza)

        means.append(img_tensor.mean(dim=(1, 2)))  # Calcola e aggiungi la media per ogni canale (RGB) alla lista delle medie (1=h, 2=w)
        stds.append(img_tensor.std(dim=(1, 2)))    # Calcola e aggiungi la deviazione standard per ogni canale (RGB) alla lista delle deviazioni standard
    
    mean = torch.stack(means).mean(dim=0)  # Combina tutte le medie delle immagini e calcola la media finale per ogni canale
    std = torch.stack(stds).mean(dim=0)    # Combina tutte le deviazioni standard delle immagini e calcola la deviazione standard finale per ogni canale
    
    return mean.tolist(), std.tolist()  # Converte i tensori risultanti in liste e li restituisce

In [6]:
# Path alle immagini nella cartella "training/images"
image_paths = glob('training/images/*.tif')  

# Esegui la computazione su GPU se disponibile
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

mean, std = compute_mean_std(image_paths)

print(f'Media dei canali RGB: {mean}')
print(f'Deviazione standard dei canali RGB: {std}')
    

Media dei canali RGB: [82.15845489501953, 110.30464935302734, 126.20238494873047]
Deviazione standard dei canali RGB: [27.315223693847656, 30.465124130249023, 38.4728889465332]


Elaborazione label:
La gestione dell'eccezione: ValueError è dovuta alle immagini e label native del dataset, in pratica in alcune immagini satellitari, c'è la presenza di nuvole, oppure rumore di qualche altro tipo. Per gestire questo rumore è stato creato un 4 canale, che effettua una maschera coprente  sulle zone dove c'è rumore/nuvole. Ci sono anche alcune labels apposite per le maschere. In casi eccezzionali l'intera immagine ha la maschera, dove avviene il mascheramento, le labels ordinarie sono prive di dati geojson, quindi se l'intera immagine è mascherata alcune label sono vuote e quindi cioò comporta quell'errore, che vista l'assenza di label, ho deciso di gestire la problematica semplicemente creando una maschera nera. 

In [25]:
# Definisce una funzione per creare una maschera binaria da geometrie
def create_mask(geojson, img_shape, transform):
    shapes = [(geom, 1) for geom in geojson.geometry if geom.is_valid and not geom.is_empty]
    if not shapes:
        raise ValueError('No valid geometry objects found for rasterize')
    mask = rasterize(shapes=shapes, out_shape=img_shape, transform=transform, fill=0, dtype=np.uint8)
    return mask

# Salva la maschera binaria normalizzata per visualizzazione
def save_normalized_mask(mask, output_path, crs, transform):
    mask_normalized = (mask * 255).astype(np.uint8)
    with rasterio.open(
        output_path,
        'w',
        driver='GTiff',
        height=mask_normalized.shape[0],
        width=mask_normalized.shape[1],
        count=1,
        dtype=mask_normalized.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(mask_normalized, 1)

def process_dataset(image_dir, label_dir, output_dir):
    image_paths = sorted(glob(os.path.join(image_dir, '*.tif')))
    label_paths = sorted(glob(os.path.join(label_dir, '*.geojson')))
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    processed_count = 0

    for img_path, label_path in zip(image_paths, label_paths):
        labels_match = gpd.read_file(label_path)

        with rasterio.open(img_path) as src:
            img_shape = (src.height, src.width)
            transform = src.transform
            img_crs = src.crs

        try:
            mask_labels_match = create_mask(labels_match, img_shape, transform)
        except ValueError as e:
            print(f"Empty or invalid label, noise on image: {img_path} due to error: {e}")
            mask_labels_match = np.zeros(img_shape, dtype=np.uint8)  # Crea una maschera nera in caso di errore
        
        img_name = os.path.basename(img_path)
        mask_name = img_name.replace('.tif', '_label_mask.tif')
        mask_output_path = os.path.join(output_dir, mask_name)

        save_normalized_mask(mask_labels_match, mask_output_path, img_crs, transform)
        
        processed_count += 1

        if processed_count % 100 == 0:
            print(f'Processed: {processed_count} masks')

Training

In [ ]:
image_dir = 'training/images'
label_dir = 'training/labels/labels_match'
output_dir = 'training/mask_from_label'

process_dataset(image_dir, label_dir, output_dir)

Validation

In [26]:
image_dir = 'validation/images'
label_dir = 'validation/labels/labels_match'
output_dir = 'validation/mask_from_label'

process_dataset(image_dir, label_dir, output_dir)

Empty or invalid label, noise on image: validation/images/global_monthly_2018_10_mosaic_L15-0457E-1135N_1831_3648_13.tif due to error: No valid geometry objects found for rasterize
Processed: 100 masks
Empty or invalid label, noise on image: validation/images/global_monthly_2019_07_mosaic_L15-0457E-1135N_1831_3648_13.tif due to error: No valid geometry objects found for rasterize
Empty or invalid label, noise on image: validation/images/global_monthly_2019_08_mosaic_L15-0457E-1135N_1831_3648_13.tif due to error: No valid geometry objects found for rasterize
Empty or invalid label, noise on image: validation/images/global_monthly_2019_11_mosaic_L15-0457E-1135N_1831_3648_13.tif due to error: No valid geometry objects found for rasterize
Processed: 200 masks


Test

In [23]:
image_dir = 'test/images'
label_dir = 'test/labels/labels_match'
output_dir = 'test/mask_from_label'

process_dataset(image_dir, label_dir, output_dir)

Empty/invalid label, noise on image: test/images/global_monthly_2018_03_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geometry objects found for rasterize
Empty/invalid label, noise on image: test/images/global_monthly_2018_04_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geometry objects found for rasterize
Empty/invalid label, noise on image: test/images/global_monthly_2018_05_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geometry objects found for rasterize
Empty/invalid label, noise on image: test/images/global_monthly_2018_06_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geometry objects found for rasterize
Empty/invalid label, noise on image: test/images/global_monthly_2018_07_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geometry objects found for rasterize
Empty/invalid label, noise on image: test/images/global_monthly_2018_08_mosaic_L15-1848E-0793N_7394_5018_13.tif due to error: No valid geome